# Assigment 4

In [44]:
import math
import numpy as np
import numpy.matlib
import scipy.sparse as sp
import matplotlib.pyplot as plt
import pandas as pd

import igl
import meshplot as mp

from math import sqrt

# Reading the Triangle Mesh and Constraint Data 

In [13]:
v, f = igl.read_triangle_mesh("data/irr4-cyl2.off")
tt, _ = igl.triangle_triangle_adjacency(f)

c = np.loadtxt("data/irr4-cyl2.constraints")
cf = c[:, 0].astype(np.int64)
c = c[:, 1:]

# Function that computes the Vector Field using Soft Constraints

In [14]:
def align_field(V, F, TT, soft_id, soft_value, llambda):
    assert(soft_id[0] > 0)
    assert(soft_id.shape[0] == soft_value.shape[0])

    
    # Edges
    e1 = V[F[:, 1], :] - V[F[:, 0], :]
    e2 = V[F[:, 2], :] - V[F[:, 0], :]

    # Compute the local reference systems for each face, T1, T2
    T1 = e1 / np.linalg.norm(e1, axis=1)[:,None]
        
    T2 =  np.cross(T1, np.cross(T1, e2))
    T2 /= np.linalg.norm(T2, axis=1)[:,None]
  
    # Arrays for the entries of the matrix
    data = []
    ii = []
    jj = []
    
    index = 0
    for f in range(F.shape[0]):
        for ei in range(3): # Loop over the edges
            
            # Look up the opposite face
            g = TT[f, ei]
            
            # If it is a boundary edge, it does not contribute to the energy
            # or avoid to count every edge twice
            if g == -1 or f > g:
                continue
                
            # Compute the complex representation of the common edge
            e  = V[F[f, (ei+1)%3], :] - V[F[f, ei], :]
            
            vef = np.array([np.dot(e, T1[f, :]), np.dot(e, T2[f, :])])
            vef /= np.linalg.norm(vef)
            ef = (vef[0] + vef[1]*1j).conjugate()
            
            veg = np.array([np.dot(e, T1[g, :]), np.dot(e, T2[g, :])])
            veg /= np.linalg.norm(veg)
            eg = (veg[0] + veg[1]*1j).conjugate()
            
            
            # Add the term conj(f)^n*ui - conj(g)^n*uj to the energy matrix
            data.append(ef);  ii.append(index); jj.append(f)
            data.append(-eg); ii.append(index); jj.append(g)

            index += 1
            
    
    sqrtl = sqrt(llambda)
    
    # Convert the constraints into the complex polynomial coefficients and add them as soft constraints
    
    # Rhs of the system
    b = np.zeros(index + soft_id.shape[0], dtype=complex)
    
    for ci in range(soft_id.shape[0]):
        f = soft_id[ci]
        v = soft_value[ci, :]
        
        # Project on the local frame
        c = np.dot(v, T1[f, :]) + np.dot(v, T2[f, :])*1j
        
        data.append(sqrtl); ii.append(index); jj.append(f)
        b[index] = c * sqrtl
        
        index += 1
    
    assert(b.shape[0] == index)
    
    
    # Solve the linear system
    A = sp.coo_matrix((data, (ii, jj)), shape=(index, F.shape[0])).asformat("csr")
    u = sp.linalg.spsolve(A.H @ A, A.H @ b)
    
    R = T1 * u.real[:,None] + T2 * u.imag[:,None]

    return R

# Plot of Traingle Mesh with Vector Field and Soft Constraint 

In [15]:
def plot_mesh_field(V, F, R, constrain_faces):
    # Highlight in red the constrained faces
    col = np.ones_like(f)
    col[constrain_faces, 1:] = 0
    
    # Scaling of the representative vectors
    avg = igl.avg_edge_length(V, F)/2

    #Plot from face barycenters
    B = igl.barycenter(V, F)

    p = mp.plot(V, F, c=col)
    p.add_lines(B, B + R * avg)
    
    return p

In [16]:
R_soft = align_field(v, f, tt, cf, c, 1e6)
p = plot_mesh_field(v, f, R_soft, cf)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

# Function that computes the Vector Field using Hard Constraints

In [17]:
def align_field(V, F, TT, constraint_id, constraint_value):
    assert(constraint_id[0] > 0)
    assert(constraint_id.shape[0] == constraint_value.shape[0])

    
    # Edges
    e1 = V[F[:, 1], :] - V[F[:, 0], :]
    e2 = V[F[:, 2], :] - V[F[:, 0], :]

    # Compute the local reference systems for each face, T1, T2
    T1 = e1 / np.linalg.norm(e1, axis=1)[:,None]
        
    T2 =  np.cross(T1, np.cross(T1, e2))
    T2 /= np.linalg.norm(T2, axis=1)[:,None]
  
    # Arrays for the entries of the matrix
    data = []
    ii = []
    jj = []
    indices = []
    
    index = 0
    for f in range(F.shape[0]):
        for ei in range(3): # Loop over the edges
            
            # Look up the opposite face
            g = TT[f, ei]
            
            # If it is a boundary edge, it does not contribute to the energy
            # or avoid to count every edge twice
            if g == -1 or f > g:
                continue
                
            # Compute the complex representation of the common edge
            e  = V[F[f, (ei+1)%3], :] - V[F[f, ei], :]
            
            vef = np.array([np.dot(e, T1[f, :]), np.dot(e, T2[f, :])])
            vef /= np.linalg.norm(vef)
            ef = (vef[0] + vef[1]*1j).conjugate()
            
            veg = np.array([np.dot(e, T1[g, :]), np.dot(e, T2[g, :])])
            veg /= np.linalg.norm(veg)
            eg = (veg[0] + veg[1]*1j).conjugate()
            
            
            # Add the term conj(f)^n*ui - conj(g)^n*uj to the energy matrix
            data.append(ef);  ii.append(index); jj.append(f)
            data.append(-eg); ii.append(index); jj.append(g)
            
            # Saves the faces that do not contain a constraint
            if (f not in constraint_id) and (f not in indices):
                indices.append(f)
            if (g not in constraint_id) and (g not in indices):
                indices.append(g)

            index += 1
            
    # Constructing Matrix Q
    A = sp.coo_matrix((data, (ii, jj)), shape=(index, F.shape[0])).asformat("csr")
    Q = (A.H@A).todense()
    
    # constructing vector b and x
    b = np.zeros([F.shape[0],1], dtype=complex)
    x = np.zeros([F.shape[0],1], dtype=complex)
    
    # Constructing the Matrices Qff, Qfc and vector bf
    for ci in range(constraint_id.shape[0]):
        f = constraint_id[ci]
        v = constraint_value[ci, :]
        # Project on the local frame
        c = np.dot(v, T1[f, :]) + np.dot(v, T2[f, :])*1j
        x[f,0] = c 
        
    xc = x[constraint_id,:]    
    Qff = Q[indices,:][:,indices]
    Qfc = Q[:,constraint_id][indices,:]
    bf = b[indices,:] - Qfc@xc
    
    # Solving the linear system
    x[indices] = sp.linalg.spsolve(Qff, bf).reshape((len(indices),1))
    
    # Transforming back to 3D Coordinates
    R = T1 * x[:,0].real[:,None] + T2 * x[:,0].imag[:,None]

    return R

# Plot of Traingle Mesh with Vector Field and Hard Constraints

In [45]:
R_hard = align_field(v, f, tt, cf, c)
filename = 'vector_field_with_hard_constraints'
np.savetxt(filename, R_hard)
p = plot_mesh_field(v, f, R_hard, cf)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

# Reconstructing a scalar field from a vector field

We to find a scalar function S(x) defined over the surface whose gradient fits a given vector field as closely as possible. The scalar field is defined by values on the mesh vertices that are linearly interpolated over each triangle's interior: for given vertex values si, the function S(x) inside a triangle t is computed as 

$$
S_t(x) = \sum_{vertices \, i \, \in \, t}^{3} s_i \, \phi_i^t(x)
$$

where $\phi_i^t(x)$ are the linear "hat" functions associated with each triangle vertex. Then the scalar function's gradient is 

$$
g_t(x) = \sum_{vertices \, i \, \in \, t}^{3}s_i \, \nabla \phi_i^t(x)
$$

Since the "hat" functions are piecewise linear, their gradients are constant within each triangle, and so is $g_t$. Specifically, $g_t$ is a linear combination of the constant hat function gradients with the (unknown) values $s_i$ as coefficients, meaning that we can write an expression of the form $g = G\,s$.

Since there is no guarantee that our interpolated face-based field is actually the gradient of some function, we cannot attempt to integrate it directly. Instead, we will try to find S(x) by asking its gradient to approximate the vector field u in the least-squares sense: 

$$
\begin{align*}
\min \sum_{face \, t} A_t \mid g_t - u_t \mid^2 &= \min  A\ \mid G \, s - u \, \mid^2 \\
&= \min \,(G \, s - u)^T \, A \, (G \, s - u) \\
&= \min \, (s^T \, G^T \,A \, G \, s - s^T \, G^T \, A \, u - u^T \, A \, G \, s + u^T \,A \, u) \\
&= \min \, (s^T \, G^T \, A \, G \, s - 2 \, s^T \, G^T \, A \, u + u^T \, A \, u) \\
&= \min (s^T \, K \, s + \, s^T \, b + c) \\
\end{align*}
$$

where $A_t$ is triangle $t$'s area, $g_t$ is the (unknown) function gradient on the triangle, and $u_t$ is the triangle's vector assigned by the guiding vector field. Therefore, we have that

$$
\begin{align*}
K \, s +  b = 0 \\
K \, s = -b
\end{align*}
$$
where $K = G^T \, A\, G$ and $b = - \, G^T \, A \, u$

In [46]:
# Calcalating Triangle Areas A_t
A_t = igl.doublearea(v,f)
A_t = np.matlib.repmat(A_t, 1, 3)
n = A_t.shape[1]
A_t = A_t.reshape((n,))
A = sp.coo_matrix(np.diag(A_t))

# Calcalating Gradient Matrix G
G = igl.grad(v,f)

K = G.T@(A@G)
R1 = R_hard.copy()
Rx = R1[:,0]
Ry = R1[:,1]
Rz = R1[:,2]
x = np.concatenate((Rx,Ry,Rz))
b = -(G.T@(A@x))

s = sp.linalg.spsolve(K, -b)
S = (G@s)
S = np.reshape(S,f.shape,order="F")

# Saving scalar field 
filename = 'scalar_field_values'
np.savetxt(filename, s)

In [28]:
def plot_mesh_with_scalar_field(V, F, S_f, s_v):
    # Visualization of computed scalar function and its gradient.
    # Scaling of the representative vectors
    S_f_mag = np.linalg.norm(S_f, axis=1)
    avg = igl.avg_edge_length(V, F) / 2

    #Plot from face barycenters
    B = igl.barycenter(V, F)

    # Plot
    q = mp.plot(V, F, s_v)
    q.add_lines(B, B + avg * S_f, shading={"line_color": "black"})

In [29]:
# Plot of Triangle mesh with Scalar field
plot_mesh_with_scalar_field(v, f, S, s)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [30]:
# Gaussian Approximation Error
Err = np.linalg.norm(R_hard-S)
Err

7.3759924243369746

# Harmonic and LSCM Parameterizations

For this task, we will experiment with flattening a mesh with a boundary onto the plane using two parameterization methods: harmonic and Least Squares Conformal (LSCM) parameterization. In both cases, two scalar fields, U and V, are computed over the mesh. The per-vertex (u, v) scalars defining these coordinate functions determine the vertices' flattened positions in the plane (the flattening is linearly interpolated within each triangle).

<h3> Harmonic Parametrization </h3>
For the harmonic parametrization example, you will first map the mesh boundary to a unit circle in the UV plane centered at the origin. The boundary U and V coordinates are then "harmonically interpolated" into the interior by solving the Laplace equation with Dirichlet boundary conditions (setting the Laplacian of U equal to zero at each interior vertex, then doing the same for V). This involves two separate linear system solves (each with the same system matrix)

In [31]:
# Reading the data
v, f  = igl.read_triangle_mesh("data/camel_head.off")

# Find the open boundary
bnd = igl.boundary_loop(f)

# Map the boundary to a circle, preserving edge proportions
bnd_uv = igl.map_vertices_to_circle(v, bnd)

# Harmonic parametrization for the internal vertices
uv = igl.harmonic_weights(v, f, bnd, bnd_uv, 1)
v_p = np.hstack([uv, np.zeros((uv.shape[0],1))])

p = mp.subplot(v, f, uv=uv, shading={"wireframe": False, "flat": False}, s=[1, 2, 0])
mp.subplot(v_p, f, uv=uv, shading={"wireframe": True, "flat": False}, s=[1, 2, 1], data=p)


In [32]:
# Plot of Triangle Mesh with Scalar Field using Harmonic Parametrization

# Calculating the gradient
u_field = uv[:,0]
G = igl.grad(v,f)
grad = G@u_field
grad = grad.reshape(f.shape,order="F")
plot_mesh_with_scalar_field(v, f, grad, u_field)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.9967555…

<h3>LSCM Parameterizations</h3>

In LSCM, the boundary is free, with the exception of two vertices that must be fixed at two different locations in the UV-plane (to pin down a global position, rotation, and scaling factor). These vertices can be chosen arbitrarily. The process is again a linear system solve, but in this case the U and V functions are entwined into a single linear system.

In [33]:
# Fix two points on the boundary
b = np.array([2, 1])

bnd = igl.boundary_loop(f)
b[0] = bnd[0]
b[1] = bnd[int(bnd.size / 2)]

bc = np.array([[0.0, 0.0], [1.0, 0.0]])

# LSCM parametrization
_, uv = igl.lscm(v, f, b, bc)

p = mp.subplot(v, f, uv=uv, shading={"wireframe": False, "flat": False}, s=[1, 2, 0])
mp.subplot(uv, f, uv=uv, shading={"wireframe": True, "flat": False}, s=[1, 2, 1], data=p)

In [34]:
# Plot of Triangle Mesh with Scalar Field using LSCM Parametrization

# Calculating the gradient
v_field = uv[:,1]
G = igl.grad(v,f)
grad = G@v_field
grad = grad.reshape(f.shape,order="F")
plot_mesh_with_scalar_field(v, f, grad, v_field)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.9967555…

# Editing a parameterization with vector fields

<h3>Editing the parameterization</h3>

In [41]:
v, f = igl.read_triangle_mesh("data/irr4-cyl2.off")

# Find the open boundary
bnd = igl.boundary_loop(f)

# Map the boundary to a circle, preserving edge proportions
bnd_uv = igl.map_vertices_to_circle(v, bnd)

# Harmonic parametrization for the internal vertices
uv = igl.harmonic_weights(v, f, bnd, bnd_uv, 1)

p = mp.subplot(v, f, uv=uv, shading={"wireframe": False}, s=[1, 2, 0])
mp.subplot(uv, f, uv=uv, shading={"wireframe": True, "flat": False}, s=[1, 2, 1], data=p)

<h3>Detecting problems with the parameterization</h3>

In [54]:
uv_new = uv.copy()
uv_new[:,1] = s
v_p = np.hstack([uv_new, np.zeros((uv_new.shape[0],1))])

# Computing Normals
normals = igl.per_face_normals(v_p, f, np.array([0.0, 0.0, 0.0]))
flip_face_index = np.where(normals[:,-1] >= 0.0)[0]
non_flip_face_index = np.where(normals[:,-1] < 0.0)[0]
col = np.ones_like(f)
col[flip_face_index, 1:] = 0
col[non_flip_face_index, 1:] = 1

p = mp.subplot(v, f, uv=uv_new, shading={"wireframe": False}, s=[1, 2, 0])
mp.subplot(uv_new, f, c=col, shading={"wireframe": True}, s=[1, 2, 1], data=p)

In [58]:
# Saving flipped triangles to file
filename = 'flipped_triangles'
np.savetxt(filename, f[flip_face_index])